#### Install Selenium & Import Necessary Packages

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#### Connect Selenium to Chrome and Direct to Desired URL

In [2]:
browser = webdriver.Chrome(executable_path  = '/Users/stephaniekendall/Library/Application Support/Google/chromedriver')

In [3]:
browser.get('https://www.waterqualitydata.us/portal/#countrycode=US&sampleMedia=Water&sampleMedia=water&startDateLo=01-01-2014&startDateHi=01-01-2019&mimeType=csv&dataProfile=biological')

#### Create a Function for Downloading State CSVs

In [4]:
def get_csv(x):
    #ensure all location parameters are clear
#     clear_field = browser.find_element_by_xpath('//*[@title="Remove all items"]')
#     clear_field.click()
    
# #     # input US as Country parameter
#     country = browser.find_element_by_xpath('//*[@class="select2-search__field"]')
#     country.send_keys('US', Keys.ENTER)
    
    # input x as State parameter
    element = browser.find_element_by_xpath("//input[@placeholder='All'][1]")
    element.send_keys(x, Keys.ENTER)
    
    # Download Biological metadata sample results
    results_button = browser.find_element_by_id("biosamples")
    results_button.click()
    browser.implicitly_wait(5)
    
    # submit and download results 
    download_button = browser.find_element_by_xpath('//*[@type="submit"]')
    download_button.click()
    
    # wait until query loads to press download
    browser.implicitly_wait(15)
    continue_button = browser.find_element_by_xpath('//*[@id="progress-continue-btn"]')
    continue_button.click()

In [6]:
get_csv('IL')

#### Unzip Files

In [59]:
from zipfile import ZipFile

def unzipfile(x):
    with ZipFile(x, 'r') as zip:
        zip.printdir()
        zip.extractall()

In [65]:
unzipfile('biologicalresult1.zip')

File Name                                             Modified             Size
biologicalresult.csv                           2019-11-12 16:39:30    176633921


#### Clean Data

In [1]:
import pandas as pd

In [2]:
AZdf = pd.read_csv('AZ_water_data.csv')
AZdf.head()

/Users/stephaniekendall/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5,9,10,11,12,13,14,15,17,19,20,21,23,25,26,32,33,54,55,56,58,59,60,61,62,63,66,67,68,69,70,71,72,73,75,76,78,80,82,83,84,85,87,91,94,97,99,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,148) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,OrganizationIdentifier,OrganizationFormalName,ActivityIdentifier,ActivityTypeCode,ActivityMediaName,ActivityMediaSubdivisionName,ActivityStartDate,ActivityStartTime/Time,ActivityStartTime/TimeZoneCode,ActivityEndDate,...,LabSamplePreparationMethod/MethodQualifierTypeName,LabSamplePreparationMethod/MethodDescriptionText,PreparationStartDate,PreparationStartTime/Time,PreparationStartTime/TimeZoneCode,PreparationEndDate,PreparationEndTime/Time,PreparationEndTime/TimeZoneCode,SubstitutionDilutionFactorNumeric,ProviderName
0,USGS-AZ,USGS Arizona Water Science Center,nwisaz.01.01400049,Sample-Routine,Water,Surface Water,2014-02-26,10:20:00,MST,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NWIS
1,USGS-AZ,USGS Arizona Water Science Center,nwisaz.01.01400049,Sample-Routine,Water,Surface Water,2014-02-26,10:20:00,MST,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NWIS
2,USGS-AZ,USGS Arizona Water Science Center,nwisaz.01.01400049,Sample-Routine,Water,Surface Water,2014-02-26,10:20:00,MST,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NWIS
3,USGS-AZ,USGS Arizona Water Science Center,nwisaz.01.01400049,Sample-Routine,Water,Surface Water,2014-02-26,10:20:00,MST,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NWIS
4,USGS-AZ,USGS Arizona Water Science Center,nwisaz.01.01400049,Sample-Routine,Water,Surface Water,2014-02-26,10:20:00,MST,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NWIS


In [3]:
AZdf = AZdf[['ActivityIdentifier',
               'ActivityLocation/LatitudeMeasure', 
                'ActivityLocation/LongitudeMeasure',
               'AnalysisStartDate','CharacteristicName', 
                'ResultMeasureValue', 'ResultMeasure/MeasureUnitCode']]

In [4]:
AZdf.rename(columns={'ActivityLocation/LatitudeMeasure': 'ActivityLatitude',
                       'ActivityLocation/LongitudeMeasure': 'ActivityLongitude',
                       'CharacteristicName':'Contaminant',
                       'ResultMeasureValue':'DetectedLevel',
                       'ResultMeasure/MeasureUnitCode':'LevelUnit'}, inplace=True)

In [5]:
rows = ['Lead','Copper']
AZdf1 = AZdf[AZdf['Contaminant'].isin(rows)]

In [6]:
AZdf1 = AZdf1.dropna()

In [7]:
AZdf1 = AZdf1.replace(to_replace ='ND', value='0')

In [8]:
AZdf1['DetectedLevel'] = AZdf1.DetectedLevel.astype(float)

In [44]:
AZdf1

,ActivityIdentifier,ActivityLatitude,ActivityLongitude,AnalysisStartDate,Contaminant,DetectedLevel,LevelUnit
719156,BVR-STK1834763-001,38.273080,-120.910100,2018-04-19,Lead,0.0015,mg/l
994991,CALWR_WQX-BL15B801331,38.302230,-122.278450,2015-06-04,Copper,0.0000,mg/l
994993,CALWR_WQX-BL15B801331,38.302230,-122.278450,2015-06-04,Lead,0.0000,mg/l
995018,CALWR_WQX-BL15B801342,38.302230,-122.278450,2015-06-04,Lead,0.0000,mg/l
995037,CALWR_WQX-BL15B801342,38.302230,-122.278450,2015-06-04,Copper,0.0010,mg/l
995087,CALWR_WQX-BL15B801375,38.367255,-122.304954,2015-06-04,Copper,0.0000,mg/l
995106,CALWR_WQX-BL15B801375,38.367255,-122.304954,2015-06-04,Lead,0.0000,mg/l
995128,CALWR_WQX-BL15B801386,38.367255,-122.304954,2015-06-04,Lead,0.0000,mg/l
995139,CALWR_WQX-BL15B801386,38.367255,-122.304954,2015-06-04,Copper,0.0050,mg/l
995164,CALWR_WQX-BL15B801701,38.367255,-122.304954,2015-06-04,Lead,0.0000,mg/l


#### Connect to MYSQL and Insert Data

In [9]:
import mysql.connector
import config

In [10]:
#set up mysql connection
cnx = mysql.connector.connect(
   host = config.host,
   user = config.user,
   password = config.password)
cursor = cnx.cursor()

In [11]:
#specify database for connection
cnx = mysql.connector.connect(
   host = config.host,
   user = config.user,
   password = config.password,
   database = "water")
cursor = cnx.cursor()

In [16]:
TABLES = {}

TABLES['AZpractice1'] = \
'''CREATE TABLE IF NOT EXISTS AZpractice1 (
     ActivityIdentifier VARCHAR(50),
     ActivityLatitude FLOAT(10),
     ActivityLongitude FLOAT(10),
     AnalysisStartDate DATE,
     Contaminant VARCHAR(20),
     DetectedLevel FLOAT(10),
     LevelUnit VARCHAR(10)
)  ENGINE=INNODB;
'''

In [17]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table AZpractice1: OK


In [14]:
subset = AZdf1[['ActivityIdentifier', 'ActivityLatitude', 
                'ActivityLongitude', 'AnalysisStartDate',
                'Contaminant', 'DetectedLevel','LevelUnit']]
tuples = [tuple(x) for x in subset.values]

In [19]:
def insert_datas(dataframe):
    tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO AZpractice1 (ActivityIdentifier,
                                               ActivityLatitude,
                                               ActivityLongitude,
                                               AnalysisStartDate,
                                               Contaminant,
                                               DetectedLevel,
                                               LevelUnit)
                           VALUES (%s, %s, %s, %s, %s, %s, %s);"""
    cursor.executemany(insert_statement, tuples)
    cnx.commit()

In [20]:
insert_datas(AZdf1)